In [1]:
#Python 2 compatibility stuff
from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range
from six.moves import configparser
import six
if six.PY2:
    ConfigParser = configparser.SafeConfigParser
else:
    ConfigParser = configparser.ConfigParser
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from szar.derivatives import Derivs_Clustering, make_fisher

EXPNAME = 'S4-1.0-CDT'
GRIDNAME = 'grid-owl2'
VERSION = '0.6'
saveid = EXPNAME + "_" + GRIDNAME + "_v" + VERSION + '_'
DIR = "../datatest/"
TIMESTAMP = "_2018-11-05-10-26-09-EST"
DERIVSFILE = saveid + "fish_derivs" + TIMESTAMP + '.npy'
FACTORFILE = saveid + "fish_factor" + TIMESTAMP + '.npy'
config = ConfigParser()
config.optionxform=str
INIFILE = "../input/pipeline.ini"
config.read(INIFILE)

['../input/pipeline.ini']

In [2]:
ps_derivs = np.load(DIR + DERIVSFILE)
fisher_facs = np.load(DIR + FACTORFILE)

In [3]:
deriv = Derivs_Clustering(INIFILE, EXPNAME, GRIDNAME, VERSION)
deriv.instantiate_params()
params = deriv.params
ps_params = deriv.fisher_params
ps_param_index = {key:index for index,key in enumerate(ps_params.keys())}

In [4]:
params_oom = np.array([0.1, 0.1, 10, 1, 1e-9, 0.01, 1, 1, 1, 1, 1, 1, 0.1, 1, 1, 1])
fisher_mat = make_fisher(ps_derivs, fisher_facs)
params_mat = np.outer(params_oom,params_oom)
fisher_renorm = params_mat * fisher_mat

In [5]:
ps_fisher = fisher_mat[:-10]
ps_fisher = ps_fisher.T[:-10].T
ps_covmat = np.linalg.inv(ps_fisher)
#u,s,v = np.linalg.svd(fisher)
#covmat = np.matmul(v.transpose(), np.matmul(np.diag(s**-1), u.transpose()))

In [6]:
def plot_cov_ellipse(cov, xval, yval, ax=None):
    if ax is None:
        ax = plt.gca()
    xx = np.array(np.arange(360) / 180. * np.pi)
    circl = np.array([np.cos(xx),np.sin(xx)])
    Lmat = np.linalg.cholesky(cov)
    ansout = np.dot(1.52*Lmat,circl)
    ax.plot(ansout[0,:]+xval, ansout[1,:]+yval)

In [7]:
def make_ellipse(param1, param2):
    sigma_h0omch2 = covmat[ps_param_index[param1], ps_param_index[param2]]
    sigma_omch2h0 = covmat[ps_param_index[param2], ps_param_index[param1]]
    sigma_h0_sqr = covmat[ps_param_index[param1], ps_param_index[param1]]
    sigma_omch2_sqr = covmat[ps_param_index[param2], ps_param_index[param2]]
    cov_h0_omch2 = np.array([[sigma_h0_sqr, sigma_h0omch2],[sigma_omch2h0, sigma_omch2_sqr]])
    plot_cov_ellipse(cov_h0_omch2, ps_params[param1], ps_params[param2], ax=None)
    plt.show()

In [8]:
idtest = np.matmul(ps_fisher, ps_covmat)
print(f"  {np.allclose(idtest, np.identity(idtest.shape[0]),atol=1e-6)}")
np.abs(idtest - np.identity(idtest.shape[0])).max()

  False


0.0027309187388426313

In [11]:
marginalized_errors = np.sqrt(np.diag(ps_covmat)) #* params_oom[:-8]
ps_fparams = dict(ps_params)
del ps_fparams['b_wl']
del ps_fparams['b_ym']
del ps_fparams['alpha_ym']
del ps_fparams['beta_ym']
del ps_fparams['gamma_ym']
del ps_fparams['Ysig']
del ps_fparams['gammaYsig']
del ps_fparams['betaYsig']
del ps_fparams['w0']
del ps_fparams['wa']
marginaldict = {key:marginalized_errors[ps_param_index[key]] for key in ps_fparams.keys()}

In [12]:
import pickle
abundances_plus_planck = '../data/savedFisher_S4-1.0-CDT_grid-owl2_owl2_v0.7_planck_mwwcdm.pkl'
with open(abundances_plus_planck, 'rb') as pickle_file:
    abund_planck_params,abund_planck_fisher = pickle.load(pickle_file, encoding='latin1')
    
abund_planck_index = {key:index for index,key in enumerate(abund_planck_params)}

In [13]:
abund_planck_cov = np.linalg.inv(abund_planck_fisher)

In [14]:
marginalized_errors_abund_planck = np.sqrt(np.diag(abund_planck_cov))
abund_planck_marginal = {key:marginalized_errors_abund_planck[abund_planck_index[key]] for key in abund_planck_params}

In [15]:
planckfile = "../data/Feb18_FisherMat_Planck_tau0.01_lens_fsky0.6.csv"
planck_fisher = np.loadtxt(planckfile, delimiter=' ')

In [16]:
planck_cov = np.linalg.inv(planck_fisher)
#h0, ombch2 omch2, as, ns, mnu

In [17]:
from orphics.stats import FisherMatrix
planck_matfisher = FisherMatrix(planck_fisher, ['H0', 'ombh2', 'omch2', 'tau', 'As', 'ns', 'mnu'])
planck_fisher = planck_matfisher.reordered(['omch2', 'ombh2', 'H0', 'ns', 'As', 'mnu', 'tau'])

/Users/dylan/Development/orphics/orphics/stats.py:203: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.params = param_list


In [20]:
ps_newfisher = np.insert(ps_fisher, [6],  np.zeros(6).reshape(6,1), axis=1)
ps_newfisher = np.insert(ps_newfisher, [6], np.zeros(7), axis=0)

In [21]:
ps_planck_fisher = ps_newfisher + planck_fisher

In [22]:
ps_planck_cov = np.linalg.inv(ps_planck_fisher)

In [24]:
np.sqrt(np.diag(ps_planck_cov))

array([4.30665417e-04, 9.86716586e-05, 4.33268918e-01, 2.32904432e-03,
       3.76527604e-11, 4.99716710e-02, 9.29637486e-03])

In [79]:
abund_planck_mat = FisherMatrix(abund_planck_fisher, abund_planck_params)
newparamorder = ['omch2', 'ombh2', 'H0', 'ns', 'As', 'mnu', 'w0', 'wa', 'alpha_ym', 'b_ym', 'beta_ym', 'gamma_ym', 'Ysig', 'gammaYsig', 'betaYsig','tau']
abund_planck_mat = abund_planck_mat.reordered(newparamorder)
#abund_planck_mat = abund_planck_mat[:-9]
#abund_planck_mat = abund_planck_mat.T[:-9].T


In [103]:
ps_matfisher = FisherMatrix(fisher_mat, list(ps_params.keys()))
ps_matfisher = ps_matfisher[:-1]
ps_matfisher = ps_matfisher.T[:-1].T
ps_newfisherbig = np.insert(ps_matfisher.values, [15],  np.zeros(15).reshape(15,1), axis=1)
ps_newfisherbig = ps_newfisherbig = np.insert(ps_newfisherbig, [15], np.zeros(16), axis=0)

In [106]:
ps_abund_planck_fisher = ps_newfisherbig + abund_planck_mat

In [107]:
ps_abund_planck_cov = np.linalg.inv(ps_abund_planck_fisher)

In [112]:
np.sqrt(np.diag(ps_abund_planck_cov)[:-10])


array([3.91802223e-04, 1.62537844e-04, 1.57851139e-01, 2.65372174e-03,
       9.75638220e-12, 1.19849232e-02])

In [111]:
print(newparamorder)

['omch2', 'ombh2', 'H0', 'ns', 'As', 'mnu', 'w0', 'wa', 'alpha_ym', 'b_ym', 'beta_ym', 'gamma_ym', 'Ysig', 'gammaYsig', 'betaYsig', 'tau']


In [113]:
np.sqrt(np.diag(ps_planck_cov))

array([4.30665417e-04, 9.86716586e-05, 4.33268918e-01, 2.32904432e-03,
       3.76527604e-11, 4.99716710e-02, 9.29637486e-03])